# Upload the dataset on neo4j

how to connect to your database: https://neo4j.com/docs/getting-started/languages-guides/neo4j-python/

how to import data: https://neo4j.com/docs/python-manual/current/query-simple/

best practices: https://neo4j.com/developer-blog/neo4j-driver-best-practices/

In [ ]:
!pip install neo4j

In [13]:
import csv
dir_path = "data_csv/50MB/"   # 50MB  100MB  200MB

with open(dir_path + 'customers.csv') as f:
    customers = list(csv.DictReader(f))

with open(dir_path + 'terminals.csv') as f:
    terminals = list(csv.DictReader(f))

with open(dir_path + 'transactions.csv') as f:
    transactions = list(csv.DictReader(f))


In [2]:
database = "mb50"              # mb50  mb100  mb200

In [15]:
from neo4j import GraphDatabase
from datetime import datetime

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:

    driver.verify_connectivity()
    
    try:
        # Create customer nodes
        for customer in customers:
            records, summary, keys = driver.execute_query(
                """
                MERGE (c:Customer {customer_id: toInteger($customer.CUSTOMER_ID), x_customer_id: toFloat($customer.x_customer_id),y_customer_id: toFloat($customer.y_customer_id), mean_amount: toFloat($customer.mean_amount), std_amount: toFloat($customer.std_amount), mean_nb_tx_per_day: toFloat($customer.mean_nb_tx_per_day), available_terminals: $customer.available_terminals, nb_terminals: toInteger($customer.nb_terminals)})
                """,
                customer=customer,
                database_=database,
            )

        for terminal in terminals:
            records, summary, keys = driver.execute_query(
                """
                MERGE (t:Terminal {terminal_id: toInteger($terminal.TERMINAL_ID), x_terminal_id: toFloat($terminal.x_terminal_id),y_terminal_id: toFloat($terminal.y_terminal_id)})
                """,
                terminal=terminal,
                database_=database,
            )

        # Create transaction relationships
        for transaction in transactions:
            transaction['TX_DATETIME'] = datetime.strptime(transaction['TX_DATETIME'], "%Y-%m-%d %H:%M:%S")
            records, summary, keys = driver.execute_query(
                """
                MATCH (customer:Customer {customer_id: toInteger($transaction.CUSTOMER_ID)})
                MATCH (terminal:Terminal {terminal_id: toInteger($transaction.TERMINAL_ID)})
                MERGE (customer)-[:MADE_TRANSACTION {transaction_id: toInteger($transaction.TRANSACTION_ID), tx_datetime: datetime($transaction.TX_DATETIME), tx_amount: toFloat($transaction.TX_AMOUNT), tx_time_seconds: toInteger($transaction.TX_TIME_SECONDS), tx_time_days: toInteger($transaction.TX_TIME_DAYS), tx_fraud: toInteger($transaction.TX_FRAUD)}]->(terminal)
                """,
                transaction=transaction,
                database_=database,
            )

    except Exception as e:
        print(e)


KeyboardInterrupt: 

# Queries

### Query a

For each customer checks that the spending frequency and the spending amounts of the last
month is under the usual spending frequency and the spending amounts for the same
period.

In [3]:
from datetime import datetime, timedelta

from neo4j import GraphDatabase
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

# compare last month with the same month of the last year. es: gen 2024 and gen 2023
now = datetime.strptime("2024-03-08 12:00:00", "%Y-%m-%d %H:%M:%S")
start_of_current_month = datetime(now.year, now.month, 1)
end_of_last_month = start_of_current_month - timedelta(days=1)
start_of_last_month = datetime(end_of_last_month.year, end_of_last_month.month, 1)
print(start_of_last_month)
print(start_of_current_month)

end_of_last_year = start_of_current_month - timedelta(days=365)
start_of_last_year = start_of_last_month - timedelta(days=365)
print(start_of_last_year)
print(end_of_last_year)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        parameters = {
            "start_of_last_month": start_of_last_month.strftime("%Y-%m-%dT%H:%M:%S"),
            "start_of_current_month": start_of_current_month.strftime("%Y-%m-%dT%H:%M:%S"),
            
            "start_of_last_year": start_of_last_year.strftime("%Y-%m-%dT%H:%M:%S"),
            "end_of_last_year": end_of_last_year.strftime("%Y-%m-%dT%H:%M:%S"),
        }
        records, summary, keys = driver.execute_query(
            """
            MATCH (c:Customer)-[t:MADE_TRANSACTION]->()
            WHERE (t.tx_datetime >= datetime($parameters.start_of_last_month) AND t.tx_datetime < datetime($parameters.start_of_current_month)) OR (t.tx_datetime >= datetime($parameters.start_of_last_year) AND t.tx_datetime < datetime($parameters.end_of_last_year))
            WITH c,
               COUNT(CASE WHEN t.tx_datetime >= datetime($parameters.start_of_last_month) AND t.tx_datetime < datetime($parameters.start_of_current_month) THEN 1 ELSE NULL END) AS lastMonthFrequency,
               SUM(CASE WHEN t.tx_datetime >= datetime($parameters.start_of_last_month) AND t.tx_datetime < datetime($parameters.start_of_current_month) THEN toFloat(t.tx_amount) ELSE 0.0 END) AS lastMonthAmount,
               
               COUNT(CASE WHEN t.tx_datetime >= datetime($parameters.start_of_last_year) AND t.tx_datetime < datetime($parameters.end_of_last_year) THEN 1 ELSE NULL END) AS lastYearFrequency,
               SUM(CASE WHEN t.tx_datetime >= datetime($parameters.start_of_last_year) AND t.tx_datetime < datetime($parameters.end_of_last_year) THEN toFloat(t.tx_amount) ELSE 0.0 END) AS lastYearAmount
            
            RETURN c.customer_id AS customerId,
                   CASE
                      WHEN lastMonthFrequency < lastYearFrequency
                      THEN "under the usual"
                      ELSE "over the usual"
                   END AS spending_frequency,
                   CASE
                      WHEN lastMonthAmount < lastYearAmount
                      THEN "under the usual"
                      ELSE "over the usual"
                   END AS spending_amounts
            """,
            parameters=parameters,
            database_=database
        )

        # Loop on result
        for record in records:
            print(record)

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)


2024-02-01 00:00:00
2024-03-01 00:00:00
2023-02-01 00:00:00
2023-03-02 00:00:00
<Record customerId=0 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=1 spending_frequency='under the usual' spending_amounts='over the usual'>
<Record customerId=2 spending_frequency='under the usual' spending_amounts='under the usual'>
<Record customerId=3 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=4 spending_frequency='under the usual' spending_amounts='under the usual'>
<Record customerId=5 spending_frequency='over the usual' spending_amounts='under the usual'>
<Record customerId=6 spending_frequency='under the usual' spending_amounts='under the usual'>
<Record customerId=7 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=8 spending_frequency='under the usual' spending_amounts='under the usual'>
<Record customerId=9 spending_frequency='over the usual' spending_amounts='under t

Another interpretation of the question

In [84]:
from datetime import datetime, timedelta

from neo4j import GraphDatabase
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

now = datetime.strptime("2024-03-08 12:00:00", "%Y-%m-%d %H:%M:%S")
start_of_current_month = datetime(now.year, now.month, 1)
end_of_last_month = datetime(now.year, now.month, 1) - timedelta(days=1)
start_of_last_month = datetime(end_of_last_month.year, end_of_last_month.month, 1)

print(start_of_last_month)
print(start_of_current_month)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    
    driver.verify_connectivity()
    
    try:
        parameters = {
            "start_of_last_month": start_of_last_month.strftime("%Y-%m-%dT%H:%M:%S"),
            "start_of_current_month": start_of_current_month.strftime("%Y-%m-%dT%H:%M:%S"),
        }
        records, summary, keys = driver.execute_query(
            """
            MATCH (c:Customer)-[t:MADE_TRANSACTION]->()
            WHERE t.tx_datetime >= datetime($parameters.start_of_last_month) AND t.tx_datetime < datetime($parameters.start_of_current_month)
            WITH c, COUNT(t) AS lastMonthFrequency, SUM(toFloat(t.tx_amount)) AS lastMonthAmount
            RETURN c.customer_id AS customerId,
                   CASE
                      WHEN lastMonthFrequency < c.mean_nb_tx_per_day
                      THEN "under the usual"
                      ELSE "over the usual"
                   END AS spending_frequency,
                   CASE
                      WHEN lastMonthAmount < c.mean_amount
                      THEN "under the usual"
                      ELSE "over the usual"
                   END AS spending_amounts
            """,
            parameters=parameters,
            database_=database
        )

        # Loop on result
        for record in records:
            print(record)

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)


2024-02-01 00:00:00
2024-03-01 00:00:00
<Record customerId=0 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=1 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=2 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=3 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=4 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=5 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=6 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=7 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=8 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=9 spending_frequency='over the usual' spending_amounts='over the usual'>
<Record customerId=10 spending_frequency

### Query b

For each terminal identify the possible fraudulent transactions. The fraudulent transactions
are those whose import is higher than 20% of the maximal import of the transactions
executed on the same terminal in the last month.

In [5]:
from neo4j import GraphDatabase

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        records, summary, keys = driver.execute_query(
            """
            MATCH ()-[t:MADE_TRANSACTION]->(tm:Terminal)<-[fraud:MADE_TRANSACTION]-()
            WHERE t.tx_datetime >= fraud.tx_datetime - duration({months: 1}) AND t.tx_datetime < fraud.tx_datetime
            WITH tm, fraud, MAX(t.tx_amount) AS max_amount
            WHERE fraud.tx_amount > max_amount*1.2
            RETURN DISTINCT tm.terminal_id as terminalId,
                   COLLECT(fraud.transaction_id) as transactionsId
            ORDER BY tm.terminal_id
            """,
            database_=database
        )
        
        """ use this one to verify the single fraudulent transactions
            RETURN tm.terminal_id as terminalId,
                   fraud.transaction_id as transactionId,
                   fraud.tx_amount as tx_amount,
                   max_amount
        """

        # Loop on result
        for record in records:
            print(record)

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)

<Record terminalId=0 transactionsId=[3326182, 2800883, 1683182, 998022, 843219, 592770, 132179]>
<Record terminalId=1 transactionsId=[3678111, 3316724, 2993533, 1532888, 817035, 15927, 53459]>
<Record terminalId=2 transactionsId=[3306095, 3186775, 2533161, 1920914, 15252, 142461]>
<Record terminalId=3 transactionsId=[3675075, 3059042, 2312774, 1423199, 1231446, 353865, 38540, 76275, 221476]>
<Record terminalId=4 transactionsId=[3319732, 2786548, 2659876, 561077, 385503, 29602, 132242]>
<Record terminalId=5 transactionsId=[3689818, 1071000, 481254, 31609]>
<Record terminalId=6 transactionsId=[1951444, 372273, 47163, 69752, 90371]>
<Record terminalId=7 transactionsId=[3369437, 1678840, 1231261, 7710, 9973, 52957, 126007]>
<Record terminalId=8 transactionsId=[2821833, 2601750, 1495995, 967229, 157949]>
<Record terminalId=9 transactionsId=[2881874, 1747395, 16171, 41292, 72732]>
<Record terminalId=10 transactionsId=[3154015, 2442104, 1968866, 724561, 311009, 13598, 18046, 25271, 60581]>
<R

### Query c

Given a user u, determine the “co-customer-relationships CC of degree k”. A user u’ is a co-
customer of u if you can determine a chain “u1-t1-u2-t2-…tk-1-uk“ such that u1=u, uk=u’, and for
each 1<=I,j<=k, ui <> uj, and t1,..tk-1 are the terminals on which a transaction has been
executed. Therefore, CCk(u)={u’| a chain exists between u and u’ of degree k}. Please, note
that depending on the adopted model, the computation of CCk(u) could be quite
complicated. Consider therefore at least the computation of CC3(u) (i.e. the co-costumer
relationships of degree 3).

degree 3 means:
(u:Customer)-[:MADE_TRANSACTION]->(:Terminal)<-[:MADE_TRANSACTION]-(:Customer)-[:MADE_TRANSACTION]->(:Terminal)<-[:MADE_TRANSACTION]-(u3:Customer)

we count 4 MADE_TRANSACTION relationships
 

           
/* MATCH p=(c1:Customer {CUSTOMER_ID: toInteger($parameters.user_id)})-[:MADE_TRANSACTION*2]-(c2:Customer)
   WHERE c1 <> c2
   UNWIND nodes(p) as n
   WITH p, c2, count(DISTINCT n) as num
   RETURN DISTINCT c2.CUSTOMER_ID as customerId, min(num)
   LIMIT 1000
*/

In [6]:
from neo4j import GraphDatabase
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

database = "mb200"

#given a user
user_id = "390"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        parameters = {
            "user_id": user_id,
        }
        records, summary, keys = driver.execute_query(
            """
            MATCH p=(c1:Customer {customer_id: toInteger($parameters.user_id)})-[:MADE_TRANSACTION*4]-(c2:Customer)
            WHERE c1 <> c2
            RETURN DISTINCT c2.customer_id as customerId
            LIMIT 1000
            """,
            parameters=parameters,
            database_=database
        )
    

        # Loop on result
        for record in records:
            print(record)

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)

<Record customerId=4560>
<Record customerId=1848>
<Record customerId=4371>
<Record customerId=2334>
<Record customerId=2695>
<Record customerId=167>
<Record customerId=3184>
<Record customerId=4066>
<Record customerId=2157>
<Record customerId=624>
<Record customerId=3992>
<Record customerId=3326>
<Record customerId=926>
<Record customerId=1360>
<Record customerId=4552>
<Record customerId=907>
<Record customerId=4658>
<Record customerId=1545>
<Record customerId=4856>
<Record customerId=1994>
<Record customerId=1522>
<Record customerId=521>
<Record customerId=128>
<Record customerId=2427>
<Record customerId=2651>
<Record customerId=1607>
<Record customerId=3896>
<Record customerId=4164>
<Record customerId=4782>
<Record customerId=4275>
<Record customerId=1588>
<Record customerId=2923>
<Record customerId=4379>
<Record customerId=2770>
<Record customerId=3064>
<Record customerId=4212>
<Record customerId=11>
<Record customerId=3609>
<Record customerId=1858>
<Record customerId=1728>
<Record 

### Query d.i

Extend the logical model that you have stored in the NOSQL database by introducing the
following information:

  i. Each transaction should be extended with:
      1. The period of the day {morning, afternoon, evening, night} in which the
         transaction has been executed.
      2. The kind of products that have been bought through the transaction {high-
         tech, food, clothing, consumable, other}
      3. The feeling of security expressed by the user. This is an integer value
         between 1 and 5 expressed by the user when conclude the transaction.
  The values can be chosen randomly.

In [87]:
from neo4j import GraphDatabase
import random

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        # since there are too many transactions we have to split them into smaller batches
        records, summary, keys = driver.execute_query(
            """
            MATCH ()-[t:MADE_TRANSACTION]->() 
            RETURN t.transaction_id
            """,
            database_=database
        )
        
        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))
        
        transaction_ids = [record["t.transaction_id"] for record in records]
        BATCH_SIZE = 1000
        
        for i in range(0, len(transaction_ids), BATCH_SIZE):
            batch = transaction_ids[i:i+BATCH_SIZE]
            records, summary, keys = driver.execute_query(
                """
                UNWIND $batch AS trans
                WITH collect(trans) AS transactionList
                MATCH ()-[t:MADE_TRANSACTION]->()  WHERE t.transaction_id IN transactionList
                SET
                    t.period_of_day = 
                        CASE
                            WHEN datetime(t.tx_datetime).hour >= 6 AND datetime(t.tx_datetime).hour < 12 THEN "morning"
                            WHEN datetime(t.tx_datetime).hour >= 12 AND datetime(t.tx_datetime).hour < 18 THEN "afternoon"
                            WHEN datetime(t.tx_datetime).hour >= 18 AND datetime(t.tx_datetime).hour < 24 THEN "evening"
                            ELSE "night"
                        END,
                        
                    t.product_type = 
                        CASE
                          WHEN $rand < 0.2 THEN "high-tech"
                          WHEN $rand < 0.4 THEN "food"
                          WHEN $rand < 0.6 THEN "clothing"
                          WHEN $rand < 0.8 THEN "consumable"
                          ELSE "other"
                        END,
                        
                    t.security_feeling = toInteger(rand() * 5) + 1
                """,
                batch=batch,
                rand=random.random(),
                database_=database
            )

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)


The query returned 3835997 records in 18323 ms.
The query returned 0 records in 0 ms.


 ### Query d.ii
 
ii.  Customers that make more than three transactions from the same terminal
      expressing a similar average feeling of security should be connected as
      “buying_friends”. Therefore also this kind of relationship should be explicitly stored
      in the NOSQL database and can be queried. Note, two average feelings of security
      are considered similar when their difference is lower than 1.

In [88]:
from neo4j import GraphDatabase
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        # since there are too many transactions we have to split them into smaller batches
        records, summary, keys = driver.execute_query(
            """
            MATCH (c1:Customer)-[t1:MADE_TRANSACTION]->(t:Terminal)<-[t2:MADE_TRANSACTION]-(c2:Customer)
            WITH c1, c2, COUNT(DISTINCT t1) as count1, COUNT(DISTINCT t2) as count2, AVG(t1.security_feeling) as avg1, AVG(t2.security_feeling) as avg2
            WHERE c1<>c2 AND count1 > 3 AND count2 > 3 AND ABS(avg1-avg2)<1
            RETURN c1.customer_id as c1, c2.customer_id as c2
            """,
            database_=database
        )
        
        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))
        
        customers_ids = list((record["c1"], record["c2"]) for record in records)
        BATCH_SIZE = 1000
        
        for i in range(0, len(records), BATCH_SIZE):
            batch = customers_ids[i:i+BATCH_SIZE]
            records, summary, keys = driver.execute_query(
                """
                UNWIND $batch AS customers
                MATCH (c1:Customer {customer_id: toInteger(customers[0])})
                MATCH (c2:Customer {customer_id: toInteger(customers[1])})
                MERGE (c1)-[:BUYING_FRIENDS]-(c2)
                """,
                batch=batch,
                database_=database
            )
            
        # Loop on result
        for record in records:
            print(record)

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)

The query returned 592832 records in 1065412 ms.
The query returned 0 records in 0 ms.


### Query e

For each period of the day identifies the number of transactions that occurred in that period,
and the average number of fraudulent transactions.

In [170]:
from neo4j import GraphDatabase
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "12345678") 

database = "mb50"   

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        records, summary, keys = driver.execute_query(
            """
            MATCH ()-[t:MADE_TRANSACTION]->()
            WITH DISTINCT t.period_of_day as period, count(t) as num, COUNT(CASE WHEN t.tx_fraud > 0 THEN 1 END) as numFraud
            RETURN period, num, round(toFloat(numFraud)/toFloat(num)*100, 2) as fraudPERCENT
            """,
            database_=database
        )

        # Loop on result
        for record in records:
            print(record)

        print("The query returned {records_count} records in {time} ms.".format(
            records_count=len(records),
            time=summary.result_consumed_after    #result_available_after
        ))

    except Exception as e:
        print(e)

<Record period='night' num=146907 fraudPERCENT=4.43>
<Record period='morning' num=426703 fraudPERCENT=4.44>
<Record period='afternoon' num=426053 fraudPERCENT=4.47>
<Record period='evening' num=147405 fraudPERCENT=4.51>
The query returned 4 records in 672 ms.
